In [ ]:
import networkx as nx
import torch
import numpy as np

def extract_topological_features(edge_index, num_nodes, labels=None):
    """提取拓扑向量 f_u = [C(u), d(u), L(u)] [cite: 119]"""
    # 转换为 NetworkX 图进行计算
    edge_list = edge_index.t().tolist()
    G = nx.Graph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_list)
    
    # 1. 聚类系数 C(u) [cite: 116]
    clustering = nx.clustering(G)
    # 2. 节点度 d(u) [cite: 117]
    degrees = dict(G.degree())
    
    # 归一化处理
    max_deg = max(degrees.values()) if degrees else 1
    
    features = []
    for i in range(num_nodes):
        # 3. 二阶标签一致性 L(u) [cite: 118]
        # (简化实现：计算邻居中相同标签的比例)
        l_u = 0.0
        if labels is not None:
            neighbors = list(G.neighbors(i))
            if neighbors:
                matches = sum(1 for v in neighbors if labels[v] == labels[i])
                l_u = matches / len(neighbors)
        
        features.append([clustering[i], degrees[i]/max_deg, l_u])
    
    return torch.tensor(features, dtype=torch.float)